
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 7 BONUS LAB - Modular Orchestration

#### Duration: ~20 minutes

**This is an optional lab that can be completed after class if you're interested in practicing modular design. You will most likely not have time to complete this bonus lab during a live class.**


### Learning Objectives
By the end of this lab, you should be able to:
* Add a Run Job Task to your Job
* Repairing Failure Task

### Lab Overview
Now, Once we done with processing high-risk and low-risk borrowers separately, Its time to add personal details and create gold tables

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to **dbacademy** and the schema to your specific schema name shown below using the `USE` statements.
<br></br>
```
USE CATALOG dbacademy;
USE SCHEMA dbacademy.<your unique schema name>;
```

**NOTE:** The **DA** object is only used in Databricks Academy courses and is not available outside of these courses.

In [0]:
%run ./Includes/Classroom-Setup-7L

## B. Creating Starter Job

In this section, you'll programmatically create a Databricks job using the Databricks SDK.

> **Note:** The method for creating the job is defined in the [Classroom-Setup-Common]($./Includes/Classroom-Setup-Common) notebook. While the [Databricks SDK](https://databricks-sdk-py.readthedocs.io/en/latest/) is used here, a deep dive into the SDK is beyond the scope of this course.

**Instructions:**
- Run the command below to automatically create a job that includes all tasks completed up to Previous Lab.

In [0]:
DA.lesson_7_starter_job()

## C. Explore the Task Files

So far, You have Ingested, transformed and added conditional Tasks into bank loan data. Now we are going to add more transformation and add personal details into **high_risk_borrowers_silver** and **low_risk_borrowers_silver** table using **borrower_details_bronze** table.


Please find the notebooks for this lab in **Task Files** > **Lesson 7 Files**. Use the links below to view and explore the code for each task:

- [Task Files/Lesson 7 Files/7.1 - Adding Personal Details on high risk borrower table]($./Task Files/Lesson 7 Files/7.1 - Adding Personal Details on high risk borrower table)

- [Task Files/Lesson 7 Files/7.2 - Adding Personal Details on low risk borrower table]($./Task Files/Lesson 7 Files/7.2 - Adding Personal Details on low risk borrower table)

## D. Creating a Run Job

You will create a separate run job by following these steps:

1. Right-click on **Jobs and Pipelines** in the left navigation bar and open the link in a new tab.
2. Click **Create** and select **Job** from the drop-down menu.
3. Name your job as **Lab_07_Run_Job**
3. Click on **Notebook** task from recommended task and configure the notebook task as shown below:

| Setting      | Instructions |
|--------------|--------------|
| Task name    | Enter **run_job_notebook_task** |
| Type         | Select **Notebook** |
| Source       | Choose **Workspace** |
| Path         | Use the navigator to specify the path [Task Files/Lesson 7 Files/7.2 - Adding Personal Details on low risk borrower table]($./Task Files/Lesson 7 Files/7.2 - Adding Personal Details on low risk borrower table) |
| Compute      | Choose **Serverless** |

![Lesson07_run_job](./Includes/images/Lesson07_run_job.png)


## E. Adding New Tasks to the Master Job

Next, you will add both a notebook task and a run job task to your master job.

#### E1. Adding a Notebook Task

1. Go to all jobs from **Jobs & Pipelines** and select your master job named **Lab_07<-your schema name->**.

2. Click **Add Task**, select **Notebook**, and configure the task as follows:

| Setting      | Instructions |
|--------------|--------------|
| Task name    | Enter **creating_high_risk_borrower_gold_table** |
| Type         | Select **Notebook** |
| Source       | Select **Workspace** |
| Path         | Use the navigator to specify the path [Task Files/Lesson 7 Files/7.1 - Adding Personal Details on high risk borrower table]($./Task Files/Lesson 7 Files/7.1 - Adding Personal Details on high risk borrower table) under **Lesson 7 Files** |
| Compute      | Choose **Serverless** |
| Depends on   | Select **processing_high_risk_borrowers** and **creating_borrower_details_table** |
| Run if dependencies | Select **All Succeeded** |
| Parameters   | Click **Add**. For key, enter **should_fail**; for value, enter **"true"** |

3. Click **Create task**.

![Lesson07_notebook_task](./Includes/images/Lesson07_notebook_task.png)

#### E2. Adding a Run Job Task

1. Right-click on **Jobs and Pipelines** in the left navigation bar and open the link in a new tab.

2. Go to the master job named **Lab_07<-your schema name->**.

3. Click on **Add Task** and Select **Run Job** and configure the task as follows:

| Setting      | Instructions |
|--------------|--------------|
| Task name    | Enter **creating_low_risk_borrower_gold_table** |
| Type         | Ensure **Run Job** is selected |
| Job       | Select **Lab_07_Run_Job** |
| Depends      | Select **processing_low_risk_borrowers** and **creating_borrower_details_table** |
| Run if dependencies | Choose **All Succeeded** |


4. Click **Create task**.


![Lesson07_full_job](./Includes/images/Lesson07_full_job.png)


5. Click **Run Now** to execute the entire job.

## F. Repairing a Run

If your master job fails at the last task, follow these steps to repair the run:

1. Go to your job **Lab_07_<-your schema name->**.
2. In the **Runs** section, locate the failed run.
3. In the graph section, click on the failed task.
![Lesson07_failed_task](./Includes/images/Lesson07_failed_task.png)

4. This will open the script snapshot page. Click **Repair run**.
5. On the right, find the **Job parameters** section and update the parameter:
   * Key: **should_fail**
   * Value: **false**
6. Click **Repair Run** to retry the run.


![Lesson07_corrected_task](./Includes/images/Lesson07_corrected_task.png)

##G. Viewing Your Job

Once your run is successful, your final master run should look like below
![Lesson07_master_job](./Includes/images/Lesson07_master_job.png)

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>